In [ ]:
#!pip install transformers
from transformers import pipeline
import torch

classifier = pipeline("zero-shot-classification", model='NbAiLab/nb-bert-base-mnli')


Downloading:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/679M [00:00<?, ?B/s]

In [ ]:
sequences = ['Copper Inside er importert av Dui Clinique AS, distribuert videre til andre aktører og solgt i flere nettbutikker i Norge og andre europeiske land. Utstyret inneholder dikobberoksid, og er markedsført som «gjenbrukbare kobbermunnbind».',
'Kom til VM med ett mål og det var medalje på sprinten. Det klarte hun mot alle odds. Sesongen har vært fryktelig vanskelig, men på en nesten utrolig måte klarte hun å toppe formen til klassisk sprinten.',
'Lagets yngste tok ansvar som ankerkvinne og sikret stafettgull. Leverer to meget gode renn individuelt også.',
'Andelen nordmenn som sier de tror på Gud, har falt fra 53 prosent til 30 prosent på 35 år. Human-Etisk Forbund mener norsk politikk er i utakt med befolkningen.',
'Andelen som ser på seg selv som personlig kristne, har gått ned fra 24 prosent til 21 prosent de siste to åra. På lang sikt har imidlertid denne andelen imidlertid holdt seg stabil og er nå på nivået med målingene på midten av 1980-tallet.',
'Unge fortvilte mennesker, mange med selvmordstanker, har det siste året tatt kontakt med Mental Helse. Pågangen har vært så stor at organisasjonen bare klarer å besvare halvparten av alle som ringer.',
'Forskning viser at ryggsmerter kan være et tidlig symptom på coronaviruset.',
'Det var etter at Ødegaard ble utelatt fra kamptroppen i Real Madrids cupkamp mot Alcoyano at ryktene virkelig begynte å røre på seg.']

In [ ]:
#The default hypothesis_template is "This text is about{}". We need to change this on a Norwegian model
#For a simple sentiment analysis. Change it to for instance "Sentimentet i denne teksten er {}."

hypothesis_template = "Dette eksempelet handler om {}."

#Some sample categories
candidate_labels = ["positiv","negativ"]

#Let us pass this to the classifier and allow for multiple labels to be selected
result = classifier(sequences, candidate_labels, hypothesis_template=hypothesis_template, multi_class=False)

#Loop through the result
for r in result:
  print(f'\n{r["sequence"]}')
  print(f'{r["scores"][0]:.2f} - {r["labels"][0]}')

In [17]:
print(inputs)
print(outputs)

{'input_ids': tensor([[  101, 31178,   117, 15127, 17835, 10124, 21610, 10112,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1]])}
BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[ 0.4443,  0.5565,  0.4153,  ...,  0.6544, -0.3149, -0.2714],
         [ 0.1193,  0.7723, -0.5131,  ...,  1.2757, -0.5227, -0.4804],
         [-0.1280,  0.2548, -0.4373,  ...,  0.9036, -1.3713,  0.2991],
         ...,
         [ 1.0404,  0.6933,  0.3160,  ...,  0.2389, -1.2675,  0.7834],
         [-0.2146,  0.5112,  0.2117,  ...,  0.3938, -0.4052,  0.2097],
         [ 0.8145, -0.3492,  0.2241,  ...,  0.7622, -0.7285,  0.5384]]],
       grad_fn=<NativeLayerNormBackward>), pooler_output=tensor([[-4.6115e-01,  9.5709e-01,  1.1441e-01,  7.1629e-01, -6.1956e-01,
         -4.7885e-01, -4.6865e-01,  3.1465e-01, -9.9995e-01,  1.0000e+00,
          1.2754e-01,  5.2426e-01, -8.5101e-01,  5.5215e-01, -6.6756e-01,
         -9.9